In [14]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential


# Create Models

## 1.0 Resnet model

In [10]:
num_classes = 2 # specifies the number of classes to be classified

# Then add the new models 

new_model = Sequential()
new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
new_model.add(Dense(num_classes, activation='softmax'))

# since first layer is already trained, restrict/freeze its retraining

new_model.layers[0].trainable = False
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_2 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


## 2.0 VGG16 Model

In [11]:
v16_model = Sequential()
#Add the VGG16 model to our new sequential
v16_model.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
#add a dense layer to this new model after the convolutions.
v16_model.add(Dense(num_classes, activation='softmax'))
#freeze the top layers of our model (vgg16 layers)
v16_model.layers[0].trainable = False
v16_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_3 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


## 3.0 Mobile net

In [15]:
mobile_net = Sequential()
mobile_net.add(MobileNet(include_top=False, pooling='avg', weights='imagenet'))
#add a dense layer to this new model after the convolutions.
mobile_net.add(Dense(num_classes, activation='softmax'))
#freeze the top layers of our model (vgg16 layers)
mobile_net.layers[0].trainable = False
mobile_net.summary()

17225924/17225924 [==============================] - 8s 0us/step
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 1024)              3228864   
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 3230914 (12.32 MB)
Trainable params: 2050 (8.01 KB)
Non-trainable params: 3228864 (12.32 MB)
_________________________________________________________________


# Compile models

In [16]:
# Compile resnet model model
new_model.compile (optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
v16_model.compile (optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
mobile_net.compile (optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


# Load Data and perfom Augmentation using data generator

In [17]:
#load the image data

#first, declare the required image size 
image_size = 224
# The image data generator is used to augment our dataset to make sure that our model does not see the same image twice. This is especially important given that we have a relatively small dataset, hence our model is prone to overfitting
#Entropic capacity of a model. How much capacity is your model allowed to store
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory('../input/rural_and_urban_photos/train', target_size=(image_size, image_size),
                                                     batch_size = 12, 
                                                     class_mode = 'categorical')
validation_generator = data_generator.flow_from_directory('../input/rural_and_urban_photos/val', target_size=(image_size, image_size),
                                                     batch_size = 20, 
                                                     class_mode = 'categorical')


Found 72 images belonging to 2 classes.


Found 20 images belonging to 2 classes.


# Fit and adjust the models

In [18]:
#fit the model to the data 
print('Fitting the Resnet model')
new_model.fit(train_generator, epochs=10, steps_per_epoch=6, validation_data=validation_generator, validation_steps=1)
print('Fitting the VGG16 model')
v16_model.fit(train_generator, epochs=10, steps_per_epoch=6, validation_data=validation_generator, validation_steps=1)
print('Fitting the Mobile net model')
mobile_net.fit(train_generator, epochs=10, steps_per_epoch=6, validation_data=validation_generator, validation_steps=1)



Fitting the Resnet model
Epoch 1/10


6/6 [==============================] - 19s 2s/step - loss: 0.5902 - accuracy: 0.7500 - val_loss: 0.0499 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 5s 781ms/step - loss: 0.0468 - accuracy: 1.0000 - val_loss: 0.0482 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 4s 760ms/step - loss: 0.0309 - accuracy: 1.0000 - val_loss: 0.0510 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 4s 759ms/step - loss: 0.0241 - accuracy: 1.0000 - val_loss: 0.0413 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 4s 756ms/step - loss: 0.0193 - accuracy: 1.0000 - val_loss: 0.0373 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 5s 775ms/step - loss: 0.0163 - accuracy: 1.0000 - val_loss: 0.0381 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 4s 765ms/step - loss: 0.0138 - accuracy: 1.0000 - val_loss: 0.0374 - val_accuracy: 1.